In [1]:
import os
import sys

PROJECT_ROOT = "/media/ttoxopeus/basic_UNet"

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    print(f"✅ Added project root to sys.path:\n   {PROJECT_ROOT}")
else:
    print(f"ℹ️ Project root already in sys.path:\n   {PROJECT_ROOT}")


from src.training.eval import evaluate
from src.models.unet import UNet
from src.pruning.model_inspect import model_to_dataframe_with_l1, get_pruning_masks_blockwise, compute_actual_prune_ratios
from src.pruning.rebuild import rebuild_pruned_unet, find_prev_conv_name

✅ Added project root to sys.path:
   /media/ttoxopeus/basic_UNet


In [2]:
# Create baseline model
model = UNet(in_ch=1, out_ch=4, features=[64, 128, 256, 512])
df = model_to_dataframe_with_l1(model)
display(df)

,Layer,Type,Shape,In Ch,Out Ch,Num Params,Mean L1,Min L1,Max L1,L1 Std,Block Ratio,Post-Prune Ratio
0,encoders.0.net.0,Conv2d,"(64, 1, 3, 3)",1.0,64.0,576,1.516938,0.812208,2.155054,0.292238,None,None
1,encoders.0.net.3,Conv2d,"(64, 64, 3, 3)",64.0,64.0,36864,12.014021,11.101892,12.664603,0.279488,None,None
2,encoders.1.net.0,Conv2d,"(128, 64, 3, 3)",64.0,128.0,73728,11.916878,11.211008,12.546581,0.280784,None,None
3,encoders.1.net.3,Conv2d,"(128, 128, 3, 3)",128.0,128.0,147456,16.969320,16.237528,17.773602,0.300179,None,None
4,encoders.2.net.0,Conv2d,"(256, 128, 3, 3)",128.0,256.0,294912,17.010971,16.253176,17.622473,0.268146,None,None
5,encoders.2.net.3,Conv2d,"(256, 256, 3, 3)",256.0,256.0,589824,23.982208,23.363308,24.790726,0.280640,None,None
6,encoders.3.net.0,Conv2d,"(512, 256, 3, 3)",256.0,512.0,1179648,23.982014,23.167824,24.865364,0.290235,None,None
7,encoders.3.net.3,Conv2d,"(512, 512, 3, 3)",512.0,512.0,2359296,33.954517,33.072205,35.001751,0.297941,None,None
8,bottleneck.net.0,Conv2d,"(1024, 512, 3, 3)",512.0,1024.0,4718592,33.944561,33.175976,34.962524,0.282277,None,None
9,bottleneck.net.3,Conv2d,"(1024, 1024, 3, 3)",1024.0,1024.0,9437184,48.011658,47.099724,48.902229,0.284973,None,None


In [3]:
ratio = 0.25

# block_ratios = {
#     # --- Encoder DoubleConvs ---
#     "encoders.0": 0.1,
#     "encoders.1": 0.2,
#     "encoders.2": 0.3,
#     "encoders.3": 0.4,

#     # --- Bottleneck ---
#     "bottleneck": 0.5,

#     # --- Decoder DoubleConvs only (skip ConvTranspose2d ones) ---
#     "decoders.1": 0.4,
#     "decoders.3": 0.3,
#     "decoders.5": 0.2,
#     "decoders.7": 0.1,
# }

block_ratios = {
    block: ratio
    for block in [
        "encoders.0",
        "encoders.1",
        "encoders.2",
        "encoders.3",
        "bottleneck",
        "decoders.1",
        "decoders.3",
        "decoders.5",
        "decoders.7",
    ]
}



masks = get_pruning_masks_blockwise(df, block_ratios, default_ratio=0.25)

Block encoders.0 | Layer encoders.0.net.0 | ratio=0.25 | threshold=1.1479
Block encoders.0 | Layer encoders.0.net.3 | ratio=0.25 | threshold=11.4926
Block encoders.1 | Layer encoders.1.net.0 | ratio=0.25 | threshold=11.5449
Block encoders.1 | Layer encoders.1.net.3 | ratio=0.25 | threshold=16.6215
Block encoders.2 | Layer encoders.2.net.0 | ratio=0.25 | threshold=16.5955
Block encoders.2 | Layer encoders.2.net.3 | ratio=0.25 | threshold=23.7202
Block encoders.3 | Layer encoders.3.net.0 | ratio=0.25 | threshold=23.5922
Block encoders.3 | Layer encoders.3.net.3 | ratio=0.25 | threshold=33.5546
Block bottleneck.net | Layer bottleneck.net.0 | ratio=0.25 | threshold=33.6226
Block bottleneck.net | Layer bottleneck.net.3 | ratio=0.25 | threshold=47.5504
Block decoders.1 | Layer decoders.1.net.0 | ratio=0.25 | threshold=47.5831
Block decoders.1 | Layer decoders.1.net.3 | ratio=0.25 | threshold=33.6307
Block decoders.3 | Layer decoders.3.net.0 | ratio=0.25 | threshold=33.5497
Block decoders.3 |

In [4]:
pruned_model = rebuild_pruned_unet(
    model,
    masks, 
    save_path="/media/ttoxopeus/basic_UNet/results/UNet_ACDC/exp1/pruned/pruned_model.pth")

🔧 Rebuilding pruned UNet architecture...
Encoder features (after pruning): [48, 96, 192, 384]
Bottleneck out_channels: 768
💾 Saved pruned model to: /media/ttoxopeus/basic_UNet/results/UNet_ACDC/exp1/pruned/pruned_model.pth
✅ UNet successfully rebuilt.


In [5]:
post_ratios = compute_actual_prune_ratios(model, pruned_model)


In [6]:
df_pruned = model_to_dataframe_with_l1(
    pruned_model,
    remove_nan_layers=True,
    block_ratios=block_ratios,
    post_prune_ratios=post_ratios
)
display(df_pruned)

,Layer,Type,Shape,In Ch,Out Ch,Num Params,Mean L1,Min L1,Max L1,L1 Std,Block Ratio,Post-Prune Ratio
0,encoders.0.net.0,Conv2d,"(48, 1, 3, 3)",1.0,48.0,432,1.528856,0.841627,2.155054,0.299294,0.25,0.25
1,encoders.0.net.3,Conv2d,"(48, 48, 3, 3)",48.0,48.0,20736,9.025334,8.436628,9.575242,0.253444,0.25,0.25
2,encoders.1.net.0,Conv2d,"(96, 48, 3, 3)",48.0,96.0,41472,8.926842,8.311604,9.510031,0.249205,0.25,0.25
3,encoders.1.net.3,Conv2d,"(96, 96, 3, 3)",96.0,96.0,82944,12.722057,12.024176,13.250491,0.266438,0.25,0.25
4,encoders.2.net.0,Conv2d,"(192, 96, 3, 3)",96.0,192.0,165888,12.751450,12.000848,13.339971,0.234877,0.25,0.25
5,encoders.2.net.3,Conv2d,"(192, 192, 3, 3)",192.0,192.0,331776,18.009483,17.466192,18.630558,0.251844,0.25,0.25
6,encoders.3.net.0,Conv2d,"(384, 192, 3, 3)",192.0,384.0,663552,17.998230,17.164581,18.773773,0.251695,0.25,0.25
7,encoders.3.net.3,Conv2d,"(384, 384, 3, 3)",384.0,384.0,1327104,25.477407,24.807701,26.284054,0.250116,0.25,0.25
8,bottleneck.net.0,Conv2d,"(768, 384, 3, 3)",384.0,768.0,2654208,25.469301,24.719696,26.303089,0.249087,0.25,0.25
9,bottleneck.net.3,Conv2d,"(768, 768, 3, 3)",768.0,768.0,5308416,36.015743,35.253475,36.864021,0.245739,0.25,0.25


In [7]:
evaluate()

🔍 Evaluating model from: /media/ttoxopeus/basic_UNet/results/UNet_ACDC/exp1/pruned/pruned_model.pth
📂 Saving results to: results/UNet_ACDC/exp1/pruned/evaluation
✅ Loaded 3000 test batches.
🚀 Running evaluation on test set...


Evaluating: 100%|██████████| 3000/3000 [00:30<00:00, 98.55it/s] 


✅ Evaluation complete!
📊 Mean Dice: 0.4634
📊 Mean IoU:  0.4488
───────────────────────────────
Background    Dice=0.9295  IoU=0.8712
RV            Dice=0.5763  IoU=0.5763
Myocardium    Dice=0.1100  IoU=0.1100
LV            Dice=0.2377  IoU=0.2377
───────────────────────────────
💾 Saved metrics to results/UNet_ACDC/exp1/pruned/evaluation/eval_metrics.json
🧾 Saved evaluation summary to results/UNet_ACDC/exp1/pruned/evaluation/eval_summary.json
